In [1]:
!rm -rf sample_data

In [ ]:
# Install PyMuPDF
!pip install pymupdf autogen rouge
!pip install groq


In [ ]:
# Import libraries
import asyncio
import fitz
import time
import re
from groq import Groq
import os
from io import BytesIO
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from rouge import Rouge
from google.colab import files


In [ ]:
# set your API key (replace with your real key)
os.environ["OPENAI_API_KEY"] = "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"

# initialize the client
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extract text from local PDF file using PyMuPDF"""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"PDF processing error: {str(e)}")
        return "Failed to process PDF."

def truncate_text(text, max_words=300):
    """Limit text length while preserving sentence boundaries"""
    words = text.split()[:max_words]
    return ' '.join(words)

def extract_sections(text):
    """Identify and extract key sections from research paper text"""
    section_patterns = {
        'abstract': r'^\s*(?:[\d.]+\s*)?Abstract\b',
        'introduction': r'^\s*(?:[\d.]+\s*)?Introduction\b',
        'implementation': r'^\s*(?:[\d.]+\s*)?(Implementation|Methodology)\b'
    }

    sections = {k: '' for k in section_patterns}
    current_section = None

    for line in text.split('\n'):
        for section, pattern in section_patterns.items():
            if re.search(pattern, line, re.IGNORECASE):
                current_section = section
                break
        if current_section in sections:
            sections[current_section] += line + '\n'

    combined = ""
    for section in ['abstract', 'introduction', 'implementation']:
        if sections[section]:
            combined += f"{section.upper()}:\n{truncate_text(sections[section])}\n\n"

    return combined or truncate_text(text)

In [ ]:
class CustomGroupChatManager(GroupChatManager):
    """Custom conversation manager for agent interactions"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.system_review = ""
        self.user_review = ""

    async def a_validate_request(self, messages, sender, config):
        if not messages:
            return self.groupchat.agent_by_name("Human")
        last_msg = messages[-1]
        if last_msg["name"] == "Human":
            content = last_msg["content"].strip().lower()
            if content.startswith(("revise", "refine")):
                return self.groupchat.agent_by_name("Reviewer")
        return self.groupchat.agent_by_name("Human")

class EnhancedEvaluationAgent(AssistantAgent):
    """ROUGE metric analysis agent"""
    def __init__(self, name, *args, **kwargs):
        super().__init__(name=name, *args, **kwargs)
        self.rouge = Rouge()

    async def analyze_reviews(self, system_review, user_review):
        try:
            scores = self.rouge.get_scores(user_review, system_review)[0]
            r1 = scores['rouge-1']['f'] * 100
            rl = scores['rouge-l']['f'] * 100

            return (
                f"ROUGE-1 F1: {r1:.1f}%\n"
                f"ROUGE-L F1: {rl:.1f}%\n\n"

            )
        except Exception as e:
            return f"Analysis Error: {str(e)}"

In [ ]:
async def process_paper(pdf_path, summarizer):
    """Process individual research paper"""
    text = extract_text_from_pdf(pdf_path)
    if text.startswith("Failed"):
        return None
    try:
        sections_text = extract_sections(text)
        return await summarizer.a_generate_reply(
            messages=[{
                "role": "user",
                "content": f"Analyze technical content:\n{sections_text}\n"
                           "Extract concise points for:\n- Key contributions\n- Methodology\n- Technical details\n- Novel aspects"
            }]
        )
    except Exception as e:
        print(f"Processing Error: {str(e)}")
        return None

In [ ]:
async def run_analysis():
    """Main execution workflow"""
    content_dir = '/content'
    if not os.path.exists(content_dir):
        os.makedirs(content_dir)

    pdf_files = []
    print("Upload at least 6 PDF files (multiple uploads allowed)")

    while len(pdf_files) < 6:
        print(f"\nCurrent PDF count: {len(pdf_files)}/6")
        if len(pdf_files) > 0:
            print("Please upload more files to reach minimum 6")
        files.upload()
        pdf_files = sorted([
            f for f in os.listdir(content_dir)
            if f.lower().endswith('.pdf')
        ])[:6]

    pdf_paths = [os.path.join(content_dir, f) for f in pdf_files[:6]]
    print("\nSelected files:")
    for i, path in enumerate(pdf_paths):
        print(f"{i+1}. {os.path.basename(path)}")

    # Configure LLM agents
    llm_config = {
        "config_list": [{
            "model": "meta-llama/llama-4-scout-17b-16e-instruct",
            "temperature": 0.3,
            "max_tokens": 1000
        }]
    }

    summarizer = AssistantAgent(
        name="Summarizer",
        system_message="Extract 3-4 concise technical points per category:\n"
                       "- Key contributions\n- Methodology\n- Technical details\n- Novel aspects",
        llm_config=llm_config
    )

    reviewer = AssistantAgent(
        name="Reviewer",
        system_message="Generate cohesive literature review of exact 500 words not less than that.",
        llm_config=llm_config
    )

    human = UserProxyAgent(
        name="Human",
        human_input_mode="ALWAYS",
        max_consecutive_auto_reply=1,
        code_execution_config={"use_docker": False}
    )

    manager = CustomGroupChatManager(
        groupchat=GroupChat(
            agents=[human, reviewer],
            messages=[],
            max_round=2
        ),
        llm_config=llm_config
    )

    # Process papers
    summaries = []
    for idx, path in enumerate(pdf_paths):
        print(f"\nProcessing paper {idx+1}/{len(pdf_paths)}")
        paper_title = os.path.basename(path)
        if summary := await process_paper(path, summarizer):
            print(f"\n[Summarizer Agent]: Technical Summary - {paper_title}:")
            print(summary)
            summaries.append(summary)
        time.sleep(3)

    # Generate literature review
    await human.a_send(
        message=f"TECHNICAL SUMMARIES:\n{'n'.join(summaries)}",
        recipient=manager,
        request_reply=True
    )
    manager.system_review = next(
        (msg["content"] for msg in manager.groupchat.messages
         if msg["name"] == "Reviewer"), ""
    )

    # Interactive refinement
    while True:
        try:
            user_input = input("\nCommand (evaluate/revise/exit): ").strip().lower()
            if not user_input:
                continue
            if user_input.startswith("exit"):
                break
            elif user_input.startswith("evaluate"):
                print("Paste your review (END to finish):")
                user_review = "\n".join(iter(input, 'END'))
                if user_review:
                    evaluator = EnhancedEvaluationAgent(
                        name="Evaluator",
                        llm_config=llm_config
                    )
                    rouge_scores = await evaluator.analyze_reviews(
                        manager.system_review,
                        user_review
                    )
                    print(rouge_scores)
            elif user_input.startswith(("revise", "refine")):
                await human.a_send(user_input, recipient=manager, request_reply=True)
                manager.system_review = next(
                    (msg["content"] for msg in manager.groupchat.messages
                     if msg["name"] == "Reviewer"), ""
                )
        except KeyboardInterrupt:
            print("nSession terminated")
            break
    print("Analysis completed")

# Run the main workflow
await run_analysis()

Upload at least 6 PDF files (multiple uploads allowed)

Current PDF count: 0/6


Saving Application_of_Prompt_Engineering_in_AIGC__Taking_Stable_Diffusion_as_an_Example.pdf to Application_of_Prompt_Engineering_in_AIGC__Taking_Stable_Diffusion_as_an_Example (1).pdf

Selected files:
1. Application_of_Prompt_Engineering_in_AIGC__Taking_Stable_Diffusion_as_an_Example (1).pdf
2. Application_of_Prompt_Engineering_in_AIGC__Taking_Stable_Diffusion_as_an_Example.pdf
3. Does_Prompt_Engineering_Help_Turkish_Named_Entity_Recognition.pdf
4. Enhancing_Language_Models_Through_Prompt_Engineering_-_A_Survey.pdf
5. Optimizing_Human-AI_Interaction_Innovations_in_Prompt_Engineering.pdf
6. Prompt_Engineering_as_Code_PEaC_an_approach_for_building_modular_reusable_and_portable_prompts.pdf

Processing paper 1/6
[autogen.oai.client: 11-01 15:19:34] {699} WARNING - Model meta-llama/llama-4-scout-17b-16e-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



[Summarizer Agent]: Technical Summary - Application_of_Prompt_Engineering_in_AIGC__Taking_Stable_Diffusion_as_an_Example (1).pdf:
Here are the extracted points:

**Key Contributions:**

1. Proposes a design framework for prompt word engineering to facilitate quick writing of prompt words for product design concepts and creative ideas.
2. Presents a method to use Stable Diffusion to rapidly achieve creative ideas and generate product design drawings.
3. Introduces a LoRA fine-tuning model to guide product style and combines ControlNet network to control generated content.

**Methodology:**

1. Utilizes Stable Diffusion as an AI painting tool to generate product design drawings based on prompt words.
2. Employs a design framework for prompt word engineering to effectively and quickly design prompt words.
3. Leverages LoRA fine-tuning model and ControlNet network to refine and control generated content.

**Technical Details:**

1. Uses Stable Diffusion, a type of diffusion model, for AI 


[Summarizer Agent]: Technical Summary - Application_of_Prompt_Engineering_in_AIGC__Taking_Stable_Diffusion_as_an_Example.pdf:
Here are the extracted points:

**Key Contributions:**

1. Proposes a design framework for prompt word engineering to facilitate quick writing of prompt words for product design concepts and creative ideas.
2. Presents a method to use Stable Diffusion to rapidly achieve creative ideas and generate product design drawings.
3. Introduces a LoRA fine-tuning model to guide product style and combines ControlNet network to control generated content.

**Methodology:**

1. Utilizes Stable Diffusion as an AI painting tool to generate product design drawings based on prompt words.
2. Employs a design framework for prompt word engineering to effectively and quickly design prompt words.
3. Leverages LoRA fine-tuning model and ControlNet network to refine and control generated content.

**Technical Details:**

1. Uses Stable Diffusion, a type of diffusion model, for AI pain


[Summarizer Agent]: Technical Summary - Does_Prompt_Engineering_Help_Turkish_Named_Entity_Recognition.pdf:
Here are the extracted points:

**Key Contributions:**

1. Investigated prompt engineering for Turkish named entity recognition (NER) using pre-trained language models.
2. Studied two prompting strategies (zero-shot and few-shot) to guide language models towards correct entity identification.
3. Demonstrated that carefully constructed prompts can achieve high accuracy on entity recognition in languages with complex morphology.
4. Evaluated the impact of prompt templates on NER performance.

**Methodology:**

1. Formulated NER as a machine reading comprehension (MRC) task.
2. Used pre-trained large language models (LLMs) to comprehend text and answer questions about entity types.
3. Constructed prompts to elicit desired entity mentions and their types from LLMs.
4. Conducted experiments on two large language models using different prompt templates.

**Technical Details:**

1. Util

...


[Summarizer Agent]: Technical Summary - Optimizing_Human-AI_Interaction_Innovations_in_Prompt_Engineering.pdf:
Here are the extracted points:

**Key Contributions:**

1. A structured methodology for prompt generation to optimize interactions with large language models (LLMs).
2. Identification of best practices for prompt design, emphasizing clarity, specificity, and contextualization.
3. Findings indicate that tailored prompts can significantly improve accuracy across various applications.
4. Advocacy for the integration of prompt engineering in high-stakes environments to promote effective and ethical communication with AI systems.

**Methodology:**

1. Comprehensive review of existing research studies on prompt engineering.
2. Analysis of the role of prompt engineering in enhancing the quality and relevance of AI-generated outputs.
3. Examination of the challenges faced in maintaining consistency and reliability in AI responses.
4. Discussion of various prompt engineering methods, 


[Summarizer Agent]: Technical Summary - Prompt_Engineering_as_Code_PEaC_an_approach_for_building_modular_reusable_and_portable_prompts.pdf:
Here are the extracted points:

**Key Contributions:**

1. Introduces Prompt Engineering as Code (PEaC), a novel approach to organizing prompts in a modular, reusable, and portable way.
2. Enables efficient and scalable prompt systems through a human-readable data serialization language.
3. Increases prompt reusability, reduces redundancy, and promotes adaptability of prompt systems.
4. Represents progress in establishing standardized and scalable engineered prompts.

**Methodology:**

1. Uses a syntax language to assemble prompts as modular components, similar to conventional programming languages.
2. Employs a human-readable data serialization language to manage prompts.
3. Evaluates the methodology through implementation in multiple LLM-driven applications.
4. Assesses enhancements in prompt management and adaptability.

**Technical Details:**


[autogen.oai.client: 11-01 15:19:57] {699} WARNING - Model meta-llama/llama-4-scout-17b-16e-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



Next speaker: Reviewer

[autogen.oai.client: 11-01 15:19:58] {699} WARNING - Model meta-llama/llama-4-scout-17b-16e-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Reviewer (to chat_manager):

Here is a 500-word literature review based on the provided points:

The rapidly evolving field of artificial intelligence (AI) has witnessed significant advancements in natural language processing (NLP) and language models. A crucial aspect of these developments is prompt engineering, which involves designing and optimizing prompts to interact with language models effectively. This literature review aims to provide a comprehensive overview of the key contributions, methodologies, and technical details of prompt engineering.

Recent studies have highlighted the importance of prompt engineering in enhancing the performance and reliability of language models. For instance, a study proposed a design framework for prompt word engineering to facilitate quick writing of prompt words for product design concepts and creative ideas (1). Another study introduced a method to use Stable Diffusion to rapidly achieve creative ideas and generate product design drawings (2)

In [6]:
!ls -l /content


total 0


In [7]:
!git rm -r --cached .
!git add "multi-agent-system-for-creating-literature-reviews.ipynb"


fatal: pathspec '.' did not match any files
fatal: pathspec 'multi-agent-system-for-creating-literature-reviews.ipynb' did not match any files
